# Exploring the Sentinel 2 STAC catalog tools

In [1]:
import geopandas as gpd
import leafmap

from pathlib import Path
from geospatial_tools import DATA_DIR
from geospatial_tools.stac import StacSearch, PLANETARY_COMPUTER
from geospatial_tools.utils import create_date_range_for_specific_period

## Visualizing Sentinel 2 tiles

Below, we load up the Sentinel 2 griding tiles so we can determine which grid most fits with our needs. 
In this demonstration, we will be looking the southern Jackson Demonstration State Forest, which
is inside the `10SJD` tile. 

In [2]:
S2_USA_GRID_FILE = DATA_DIR / "s2_grid_usa_polygon_5070.gpkg"
s2_grid = gpd.read_file(S2_USA_GRID_FILE)

m = leafmap.Map(center=[39.3, -124.16], zoom=8)
m.add_gdf(s2_grid, layer_name='s2_grid', style={"color": "red"})
m

Map(center=[39.3, -124.16], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

## Search Parameters

Now we search for our product.

In this demonstration, we want relatively recent products, and only from June or July

We are looking for Sentinel 2 products that are already pre-processed and that have 
minimal cloud cover.

We are also limiting our search to the `10SDJ` tile.



In [3]:
# Date ranges
start_year = 2023
end_year = 2024
start_month = 6
end_month = 7
date_ranges = create_date_range_for_specific_period(start_year=start_year,
                                                    end_year=end_year,
                                                    start_month_range=start_month,
                                                    end_month_range=end_month)

# Search arguments
collection = "sentinel-2-l2a"
tile_ids = ["10SDJ"]
max_cloud_cover = 1
max_no_data_value = 5
query = {"eo:cloud_cover": {"lt": max_cloud_cover}, "s2:mgrs_tile": {"in": tile_ids}}
sortby = [{"field": "properties.eo:cloud_cover", "direction": "asc"}]

# Searching for results
search_client = StacSearch(PLANETARY_COMPUTER)
search_client.search_for_date_ranges(date_ranges=date_ranges,
                                     collections=collection,
                                     query=query,
                                     sortby=sortby, limit=100)

# Selecting optimal result
search_client.filter_no_data(property_name="s2:nodata_pixel_percentage", max_no_data_value=max_no_data_value)
search_client.sort_results_by_cloud_coverage()

sorted_items = search_client.cloud_cover_sorted_results
optimal_result2 = sorted_items[0]

# Printing relevant information about
print("\nSorted results")
for item in sorted_items:
    print(f"Item: {item.id}, {item.datetime}, "
          f"Cloud cover: {item.properties['eo:cloud_cover']}, "
          f"Nodata: {item.properties['s2:nodata_pixel_percentage']}")

print(f"\nOptimal result: {optimal_result2.id}, {optimal_result2.datetime}, "
      f"Cloud cover: {optimal_result2.properties['eo:cloud_cover']}, "
      f"Nodata: {optimal_result2.properties['s2:nodata_pixel_percentage']}")

[2024-09-04 14:50:51] INFO       [MainThread][geospatial_tools.stac] Initiating STAC API search for the following date ranges : [['2023-06-01T00:00:00Z/2023-07-31T23:59:59Z', '2024-06-01T00:00:00Z/2024-07-31T23:59:59Z'] 
	Query : [{'eo:cloud_cover': {'lt': 1}, 's2:mgrs_tile': {'in': ['10SDJ']}}]
[2024-09-04 14:50:52] INFO       [MainThread][geospatial_tools.stac] Search successful for date range [2023-06-01T00:00:00Z/2023-07-31T23:59:59Z]
[2024-09-04 14:50:53] INFO       [MainThread][geospatial_tools.stac] Search successful for date range [2024-06-01T00:00:00Z/2024-07-31T23:59:59Z]
[2024-09-04 14:50:53] INFO       [MainThread][geospatial_tools.stac] Sorting results by cloud cover (from least to most)

Sorted results
Item: S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346, 2024-07-05 18:59:21.024000+00:00, Cloud cover: 0.006778, Nodata: 1.7e-05
Item: S2B_MSIL2A_20230726T185919_R013_T10SDJ_20230727T003302, 2023-07-26 18:59:19.024000+00:00, Cloud cover: 0.036048, Nodata: 1.7e-05
Item

In [4]:
bands = ["B02", "B03", "B04", "B08", "visual"]
file_base_path = Path(f"{DATA_DIR}/sentinel-2/")
best_result = search_client.download_best_cloud_cover_result(bands=bands, base_directory=file_base_path)
best_result

[2024-09-04 14:50:53] INFO       [MainThread][geospatial_tools.stac] Downloading [S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346] ...
[2024-09-04 14:50:53] INFO       [MainThread][geospatial_tools.stac] Downloading B02 from https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/10/S/DJ/2024/07/05/S2A_MSIL2A_20240705T185921_N0510_R013_T10SDJ_20240706T050346.SAFE/GRANULE/L2A_T10SDJ_A047200_20240705T190418/IMG_DATA/R10m/T10SDJ_20240705T185921_B02_10m.tif?st=2024-09-03T18%3A50%3A52Z&se=2024-09-04T19%3A35%3A52Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2024-09-04T06%3A35%3A14Z&ske=2024-09-11T06%3A35%3A14Z&sks=b&skv=2024-05-04&sig=1YeZcTKo12HXotWHwPL5omKZgQhQlmHuHvICIYEND4Y%3D
[2024-09-04 14:51:16] INFO       [MainThread][geospatial_tools.utils] Downloaded /home/francispelletier/projects/geospatial_tools/data/sentinel-2/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_B02.tif successfully.
[2024-09-04 1

In [5]:
best_result.show_asset_items()

[2024-09-04 14:53:55] INFO       [MainThread][geospatial_tools.stac] Asset list for asset [S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346] : 
	['ID: [S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346], Band: [B02], filename: [/home/francispelletier/projects/geospatial_tools/data/sentinel-2/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_B02.tif]', 'ID: [S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346], Band: [B03], filename: [/home/francispelletier/projects/geospatial_tools/data/sentinel-2/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_B03.tif]', 'ID: [S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346], Band: [B04], filename: [/home/francispelletier/projects/geospatial_tools/data/sentinel-2/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_B04.tif]', 'ID: [S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346], Band: [B08], filename: [/home/francispelletier/projects/geospatial_tools/data/sentinel-2/S2A_MSIL2A_20240705T185921_R013_T10SDJ_202

In [6]:
merged = best_result.merge_asset(base_directory=file_base_path, delete_sub_items=True)
merged

[2024-09-04 14:53:55] INFO       [MainThread][geospatial_tools.stac] Calculated a total of [7] bands
[2024-09-04 14:53:55] INFO       [MainThread][geospatial_tools.stac] 7
[2024-09-04 14:53:55] INFO       [MainThread][geospatial_tools.stac] Creating merged asset metadata
[2024-09-04 14:53:55] INFO       [MainThread][geospatial_tools.stac] Merging asset [S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346] ...
[2024-09-04 14:53:55] INFO       [MainThread][geospatial_tools.stac] Writing band image: S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346
[2024-09-04 14:53:55] INFO       [MainThread][geospatial_tools.stac] writing asset sub item band 1
[2024-09-04 14:53:55] INFO       [MainThread][geospatial_tools.stac] writing merged index band 1
[2024-09-04 14:53:57] INFO       [MainThread][geospatial_tools.stac] Writing band image: S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346
[2024-09-04 14:53:57] INFO       [MainThread][geospatial_tools.stac] writing asset sub item band 1
[

PosixPath('/home/francispelletier/projects/geospatial_tools/data/sentinel-2/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_merged.tif')

In [7]:
reprojected = best_result.reproject_merged_asset(target_projection=5070, 
                                                 base_directory=file_base_path,
                                                 delete_merged_asset=True)
reprojected

[2024-09-04 14:54:08] INFO       [MainThread][geospatial_tools.stac] Reprojecting asset [S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346] ...
[2024-09-04 14:54:08] INFO       [MainThread][geospatial_tools.stac] Creating EPSG code from following input : [5070]
[2024-09-04 14:55:23] INFO       [MainThread][geospatial_tools.stac] Reprojected file created at /home/francispelletier/projects/geospatial_tools/data/sentinel-2/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_reprojected.tif
[2024-09-04 14:55:23] INFO       [MainThread][geospatial_tools.stac] Asset location : [/home/francispelletier/projects/geospatial_tools/data/sentinel-2/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_reprojected.tif]
[2024-09-04 14:55:23] INFO       [MainThread][geospatial_tools.stac] Deleting merged asset file for [/home/francispelletier/projects/geospatial_tools/data/sentinel-2/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_merged.tif]


PosixPath('/home/francispelletier/projects/geospatial_tools/data/sentinel-2/S2A_MSIL2A_20240705T185921_R013_T10SDJ_20240706T050346_reprojected.tif')

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Path to your multiband TIFF image
image_path = str(reprojected)

# Open the image file
with rasterio.open(image_path) as src:
    # Read the desired bands (adjust band indices as needed)
    band_red = src.read(5)  # Assuming band 3 is red
    band_green = src.read(6)  # Assuming band 2 is green
    band_blue = src.read(7)  # Assuming band 1 is blue

    # Stack the bands into an RGB array
    rgb = np.dstack((band_red, band_green, band_blue))

    # Display the image using Matplotlib
    plt.imshow(rgb)
    plt.show()